In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("../input/spaceship-titanic/train.csv")
data2 = pd.read_csv("../input/spaceship-titanic/test.csv")
df = [data,data2]
data.head()
df[0].isna().sum()

In [ ]:
#data cleaning

import seaborn as sns
import math
from sklearn import preprocessing

#replacing NA cabin info
df[0]['Cabin'].fillna('C/2/P',inplace=True)
df[1]['Cabin'].fillna('C/2/P',inplace=True)

#replacing NA age info
df[0]['Age'].fillna(df[0]['Age'].mean(),inplace=True)
df[1]['Age'].fillna(df[1]['Age'].mean(),inplace=True)

#splitting cabin into various fields
df[0][['Deck','Cabin Number','Ship Side']] = (df[0]['Cabin'].str.split('/',expand=True))
df[1][['Deck','Cabin Number','Ship Side']] = (df[1]['Cabin'].str.split('/',expand=True))

#split last name from first and then assign an int value to last name to see if there is a corrilation with surviving
df[0][['First Name','Last Name']] = (df[0]['Name'].str.split(expand=True))
df[1][['First Name','Last Name']] = (df[1]['Name'].str.split(expand=True))

#removing NA values in money related columns by assuming spent money is 0
cols_to_process = ['RoomService','ShoppingMall','FoodCourt','Spa','VRDeck']
for i in df:
    for x in cols_to_process:
        i[x].fillna(0,inplace=True)


#adding a total spent column
a = df[0]['RoomService'].values
b = df[0]['ShoppingMall'].values
c = df[0]['FoodCourt'].values
d = df[0]['Spa'].values
e = df[0]['VRDeck'].values
df[0].insert(10, 'TotalSpent', a+b+c+d+e)

a = df[1]['RoomService'].values
b = df[1]['ShoppingMall'].values
c = df[1]['FoodCourt'].values
d = df[1]['Spa'].values
e = df[1]['VRDeck'].values
df[1].insert(10, 'TotalSpent', a+b+c+d+e)


#replacing missing names



#dropping cols that will not be used
PID = df[1]['PassengerId']
df[0].drop(['PassengerId'],axis=1,inplace=True)
df[1].drop(['PassengerId'],axis=1,inplace=True)

df[0].drop(['First Name'],axis=1,inplace=True)
df[1].drop(['First Name'],axis=1,inplace=True)

df[0].drop(['Cabin'],axis=1,inplace=True)
df[1].drop(['Cabin'],axis=1,inplace=True)

df[0].drop(['Name'],axis=1,inplace=True)
df[1].drop(['Name'],axis=1,inplace=True)



cols_to_process = ['VIP', 'CryoSleep', 'Destination', 'HomePlanet', 'Deck', 'Cabin Number', 'Ship Side', 'Last Name']
le = preprocessing.LabelEncoder()
for i in df:
    for col in cols_to_process:
        i[col] = le.fit_transform(i[col])


import matplotlib.pyplot as plt
#plt.figure(figsize=(16,10))
#heatmap = sns.heatmap(data.corr()[['Transported']].sort_values(by='Transported',ascending=False), vmin=-1, vmax=1, annot=True)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))
numeric_cols = df[0].select_dtypes(include=np.number).columns.tolist()
sns.heatmap(df[0][numeric_cols + ["Transported"]].corr(), annot=True, ax=ax, 
            xticklabels=numeric_cols + ["Transported"],
            yticklabels=numeric_cols + ["Transported"]);

In [ ]:
# Expenditure features
"""
exp_feats=['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
import matplotlib.pyplot as plt
# Plot expenditure features
fig=plt.figure(figsize=(10,20))
for i, var_name in enumerate(exp_feats):
    # Left plot
    ax=fig.add_subplot(5,2,2*i+1)
    sns.histplot(data=df[0], x=var_name, axes=ax, bins=30, kde=False, hue='Transported')
    ax.set_title(var_name)

    # Right plot (truncated)
    ax=fig.add_subplot(5,2,2*i+2)
    sns.histplot(data=df[0], x=var_name, axes=ax, bins=30, kde=True, hue='Transported')
    plt.ylim([0,100])
    ax.set_title(var_name)
fig.tight_layout()  # Improves appearance a bit
plt.show()
"""

In [ ]:
df[0].head()

In [ ]:
#splitting training/test data
from sklearn.model_selection import train_test_split


X_train, X_test, Y_train, Y_test = train_test_split(df[0].drop(['Transported'], axis=1), df[0]['Transported'].values)

In [ ]:
#basic nn model
import os
import cv2
import tensorflow as tf
tf.random.set_seed(15)
NNM = tf.keras.models.Sequential()
    
NNM.add(tf.keras.layers.Dense(15, activation='relu'))

NNM.add(tf.keras.layers.Dense(128, activation='relu'))

NNM.add(tf.keras.layers.Dense(64, activation='relu'))

NNM.add(tf.keras.layers.Dense(32, activation='relu'))

NNM.add(tf.keras.layers.Dense(16, activation='relu'))

NNM.add(tf.keras.layers.Dense(2, activation='softmax'))
    
NNM.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NNM.fit(X_train, Y_train, epochs=10)
NNM.predict(X_test)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, Y_train)
Y_pred = classifier.predict(X_test)
cm = confusion_matrix(Y_test, Y_pred)
svm_acc = accuracy_score(Y_test,Y_pred)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=4000).fit(X_train, Y_train)
a = model.predict(X_test)
from sklearn.metrics import accuracy_score
log_acc = accuracy_score(Y_test, a)

In [ ]:
from sklearn.naive_bayes import GaussianNB
bayes_model = GaussianNB().fit(X_train, Y_train)
naive_pred = bayes_model.predict(X_test)
naive_acc = accuracy_score(Y_test, naive_pred)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier().fit(X_train, Y_train)
knn_pred = knn_model.predict(X_test)
knn_acc = accuracy_score(Y_test, knn_pred)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest_model = RandomForestClassifier(random_state=0, n_estimators=475, class_weight='balanced').fit(X_train, Y_train)
forest_pred = forest_model.predict(X_test)
forest_acc = accuracy_score(Y_test, forest_pred)

In [ ]:
from xgboost import XGBClassifier
XGB_model = XGBClassifier(n_estimators=230, seed=0, scale_pos_weight=1.5).fit(X_train, Y_train)
XGB_pred = XGB_model.predict(X_test)
XGB_acc = accuracy_score(Y_test, XGB_pred)

In [ ]:
print('logicistic: ', log_acc)
print('naive_bayes: ', naive_acc)
print('svm: ', svm_acc)
print('knn: ', knn_acc)
print('forest: ', forest_acc)
print('XGB: ', XGB_acc)

In [ ]:
model = forest_model = RandomForestClassifier(random_state=0, n_estimators=475, class_weight='balanced').fit(X_train, Y_train)

pred_final = model.predict(df[1])

final_dict = {'PassengerId': PID, 'Transported':pred_final}

submission_csv = pd.DataFrame(final_dict)
submission_csv.to_csv('submission.csv', index=False)